In [7]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset, DatasetDict
import numpy as np

In [5]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing(as_frame=True)
housing = housing['data'].join(housing['target']).sample(1000, random_state=42)

In [9]:
np.set_printoptions(precision=2, suppress=True)
np.corrcoef(housing, rowvar=False)

array([[ 1.  , -0.1 ,  0.51, -0.07, -0.  , -0.02, -0.1 ,  0.  ,  0.69],
       [-0.1 ,  1.  , -0.18, -0.11, -0.33,  0.03,  0.03, -0.13,  0.1 ],
       [ 0.51, -0.18,  1.  ,  0.67, -0.06, -0.01,  0.13, -0.05,  0.26],
       [-0.07, -0.11,  0.67,  1.  , -0.04, -0.08,  0.1 , -0.  , -0.04],
       [-0.  , -0.33, -0.06, -0.04,  1.  ,  0.15, -0.11,  0.11, -0.03],
       [-0.02,  0.03, -0.01, -0.08,  0.15,  1.  , -0.17,  0.19, -0.26],
       [-0.1 ,  0.03,  0.13,  0.1 , -0.11, -0.17,  1.  , -0.92, -0.14],
       [ 0.  , -0.13, -0.05, -0.  ,  0.11,  0.19, -0.92,  1.  , -0.06],
       [ 0.69,  0.1 ,  0.26, -0.04, -0.03, -0.26, -0.14, -0.06,  1.  ]])

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class TrainingConfig():
    def __init__(self):

        self.output_dir = 'outputs'
        self.learning_rate = 3e-4
        self.lr_scheduler_type = 'cosine'
        self.warmup_ratio = 0.1
        self.fp16 = True if DEVICE == 'cuda' else False
        self.evaluation_strategy = 'epoch'
        self.per_device_train_batch_size = 128
        self.per_device_eval_batch_size =  self.per_device_train_batch_size * 2
        self.num_train_epochs = 20
        self.weight_decay = 0.01
        self.report_to = 'none'

CFG =  TrainingConfig()

In [14]:
model = 'microsoft/deberta-v3-small'
tokz = AutoTokenizer.from_pretrained(model)

def tok_func(x): return tokz(x["input"])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
housing['input'] = 'TEXT1: ' + housing.context + '; TEXT2: ' + housing.target + '; ANC1: ' + housing.anchor
ds = Dataset.from_pandas(housing)
housing_tok = ds.map(tok_func, batched=True)

AttributeError: 'DataFrame' object has no attribute 'context'

In [11]:
dds = housing.train_test_split(0.25, seed=42)

AttributeError: 'DataFrame' object has no attribute 'train_test_split'

In [ ]:
#we can do ds.map(fn, batched=true)
#ds needs column lbl

In [12]:
args = TrainingArguments(**CFG.__dict__)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model, num_labels=1)
trainer = Trainer(model, args, train_dataset=train_ds, eval_dataset=eval_ds, tokenizer=tokz)